In [14]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from datetime import timedelta

def over18(url):
    res = requests.get(url, verify=False)
    # 先檢查網址是否包含'over18'字串 ,如有則為18禁網站
    if 'over18' in res.url:
#         logger.debug("18禁網頁")
        # 從網址獲得版名
        board = url.split('/')[-2]
        load = {
            'from': '/bbs/{}/index.html'.format(board),
            'yes': 'yes'
        }
        res = requests.post('https://www.ptt.cc/ask/over18', verify=False, data=load)
    return BeautifulSoup(res.text, 'html.parser'), res.status_code


In [15]:
rs = requests.session()
res = rs.get('https://www.ptt.cc/bbs/Beauty/index.html', verify=False)
soup = BeautifulSoup(res.text, 'html.parser')
soup, _ = over18('https://www.ptt.cc/bbs/Beauty/index.html')

/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: Insecu

In [16]:
res.status_code

200

In [17]:
soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>看板 Beauty 文章列表 - 批踢踢實業坊</title>
<link href="//images.ptt.cc/bbs/v2.26/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/bbs-custom.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Beauty/index.html"><span class="board-label">看板 </span>Beauty</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>
<div 

In [18]:
page_option = soup.find(id="action-bar-container")

tag = page_option.find_all(class_="btn wide")[1]
print(tag)

link = tag.get('href')
print(link)

index = int(link[link.find('index')+5:link.find('.html')])
print(index)

<a class="btn wide" href="/bbs/Beauty/index3013.html">‹ 上頁</a>
/bbs/Beauty/index3013.html
3013


In [28]:
explore_page = 3

url_list = []
for i in range(explore_page):
    url_list.append("https://www.ptt.cc/bbs/Beauty/index{}.html".format(index-i))
print(url_list)

target_time = datetime.now() - timedelta(days=1)
print("target_time={}".format(target_time))

['https://www.ptt.cc/bbs/Beauty/index3013.html', 'https://www.ptt.cc/bbs/Beauty/index3012.html', 'https://www.ptt.cc/bbs/Beauty/index3011.html']
target_time=2019-07-29 23:40:58.470496


In [20]:
find = True

while url_list and find :
    url = url_list.pop(0)
    res = rs.get(url, verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    find = False # initial
    
    print("start crawler page {}".format(url))
    
    for r_ent in soup.find_all(class_="r-ent"):
        try:
            subject = r_ent.find('div', class_="title")
            if subject.a == None:
                print("本文已被刪除")
                print("")
                continue
            print("subject={}".format(subject.a.string))                
                
            link = r_ent.find('a')['href']
            print("link={}".format(link))

            date_li = r_ent.find("div", class_="date").string.split('/')
            print("dateli={}".format(date_li))
            
            push_cnt = r_ent.find(class_="nrec").text
            if push_cnt == "爆":
                push_cnt = 100
            elif push_cnt.find("X") > 0 :
                push_cnt = -1
            elif push_cnt == "":
                push_cnt = 0
            print("push_cnt={}".format(push_cnt))
            
            if int(date_li[0]) == target_time.month  and int(date_li[1]) == target_time.day:
                print("Need process")
                find = True # has find target day, need continue
            
            if r_ent.next_sibling:
                if r_ent.next_sibling.next_sibling:
                    next_class_type = r_ent.next_sibling.next_sibling.get('class')[0]
                    if next_class_type == "r-list-sep":
                        break

    #         print(type(r_ent))

        except Exception as e:
            print('Error={}'.format(e))
            print(r_ent)

            
print(url_list)


start crawler page https://www.ptt.cc/bbs/Beauty/index3013.html
['https://www.ptt.cc/bbs/Beauty/index3012.html', 'https://www.ptt.cc/bbs/Beauty/index3011.html']


/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [30]:
url = url_list[0]
res = rs.get(url, verify=False)
soup = BeautifulSoup(res.text, 'html.parser')
find = False # initial

print("start crawler page {}".format(url))

print(soup)

# print(soup.find_all(class_="r-ent"))

# for r_ent in soup.find_all(class_="r-ent"):
#     print(r_ent)

start crawler page https://www.ptt.cc/bbs/Beauty/index3013.html
<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>批踢踢實業坊</title>
<link href="//images.ptt.cc/bbs/v2.26/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/bbs-custom.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>
</head>
<body>
<div class="bbs-screen bbs-content">
<div class="over18-notice">
<p>本網站已依網站內容分級規定處理</p>
<p>警告︰您即將進入之看板內容需滿十八歲方可瀏覽。</p>
<p>若您尚未年滿十八歲，請點選離開。若您已滿十八歲，亦不可將本區之內容派發、傳閱、出售、出租、交給或借予年齡未滿18歲的人士瀏覽，或將本網站內容向該人士出示、播放或放映。</p>
</div>
</div>
<div class="bbs-screen bbs-content center clear">
<form action="

/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [21]:
from datetime import datetime
from datetime import timedelta

datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
print(datetime_object)

datetime_object = datetime.strptime('2019/06/19 01:59:12', '%Y/%m/%d %H:%M:%S')
print(datetime_object)

print(datetime.now() )


page_time = datetime.strptime( str(datetime.now().year) + '/06/11', '%Y/%m/%d') 


delta = datetime.now() - page_time
print(delta.days)


target_time = datetime.now() - timedelta(days=3)
print(str(page_time.month) + "/" + str(page_time.day))


2005-06-01 13:33:00
2019-06-19 01:59:12
2019-07-30 23:38:55.034857
49
6/11


In [22]:
target_time = datetime.now() - timedelta(days=1)
page_time = datetime.strptime( str(target_time.year) + '/' + str(target_time.month) + '/' +  str(target_time.day), '%Y/%m/%d') 
print(page_time)
print(target_time.date())

2019-07-29 00:00:00
2019-07-29


In [23]:
page_time.date() - target_time.date()

datetime.timedelta(0)

In [11]:
res = rs.get('https://www.ptt.cc/bbs/Beauty/M.1561320666.A.1A2.html', verify=False)
soup = BeautifulSoup(res.text, 'html.parser')
soup

/home/clarck/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>[正妹] Jihyo 朴志效 - 看板 Beauty - 批踢踢實業坊</title>
<meta content="all" name="robots"/>
<meta content="Ptt BBS 批踢踢" name="keywords"/>
<meta content="https://i.imgur.com/D98PM2w.gif
https://i.imgur.com/0b9EZTb.gif
https://i.imgur.com/syYga48.gif
https://i.imgur.com/D5MNH1i.gif
https://i.imgur.com/hBUnZl8.gif
" name="description"/>
<meta content="Ptt 批踢踢實業坊" property="og:site_name"/>
<meta content="[正妹] Jihyo 朴志效" property="og:title"/>
<meta content="https://i.imgur.com/D98PM2w.gif
https://i.imgur.com/0b9EZTb.gif
https://i.imgur.com/syYga48.gif
https://i.imgur.com/D5MNH1i.gif
https://i.imgur.com/hBUnZl8.gif
" property="og:description"/>
<link href="https://www.ptt.cc/bbs/Beauty/M.1561320666.A.1A2.html" rel="canonical"/>
<link href="//images.ptt.cc/bbs/v2.26/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.26/bbs-base.css" media="

In [13]:
main_content = soup.find("div", id="main-content")
main_content

<div class="bbs-screen bbs-content" id="main-content"><div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">kaeun421 (純良的卡卡)</span></div><div class="article-metaline-right"><span class="article-meta-tag">看板</span><span class="article-meta-value">Beauty</span></div><div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[正妹] Jihyo 朴志效</span></div><div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Mon Jun 24 04:11:04 2019</span></div>
<a href="https://i.imgur.com/D98PM2w.gif" rel="nofollow" target="_blank">https://i.imgur.com/D98PM2w.gif</a>
<div class="richcontent"><blockquote class="imgur-embed-pub" data-id="D98PM2w" lang="en"><a href="//imgur.com/D98PM2w"></a></blockquote><script async="" charset="utf-8" src="//s.imgur.com/min/embed.js"></script></div><a href="https://i.imgur.com/0b9EZTb.gif" rel="nofollow" target="_blank">https://i.imgur.

In [22]:
import os
print(os.path.isdir("./test"))

True


In [23]:
os.mkdir("./test2")

In [10]:
page_option = soup.find(id="action-bar-container")

In [11]:
page_option

<div id="action-bar-container">
<div class="action-bar">
<div class="btn-group btn-group-dir">
<a class="btn selected" href="/bbs/Beauty/index.html">看板</a>
<a class="btn" href="/man/Beauty/index.html">精華區</a>
</div>
<div class="btn-group btn-group-paging">
<a class="btn wide" href="/bbs/Beauty/index1.html">最舊</a>
<a class="btn wide" href="/bbs/Beauty/index2976.html">‹ 上頁</a>
<a class="btn wide" href="/bbs/Beauty/index2978.html">下頁 ›</a>
<a class="btn wide" href="/bbs/Beauty/index.html">最新</a>
</div>
</div>
</div>

In [12]:
for btn in page_option.find_all(class_="btn wide"):
    print(btn)
    if "‹ 上頁" in btn :
        print("!")
    

<a class="btn wide" href="/bbs/Beauty/index1.html">最舊</a>
<a class="btn wide" href="/bbs/Beauty/index2976.html">‹ 上頁</a>
!
<a class="btn wide" href="/bbs/Beauty/index2978.html">下頁 ›</a>
<a class="btn wide" href="/bbs/Beauty/index.html">最新</a>


In [13]:
tag = page_option.find_all(class_="btn wide")[1]
tag

<a class="btn wide" href="/bbs/Beauty/index2976.html">‹ 上頁</a>

In [14]:
link = tag.get('href')
link

'/bbs/Beauty/index2976.html'

In [15]:
index = int(link[link.find('index')+5:link.find('.html')])
print(index)

2976


In [59]:
r_ent.find(class_="nrec").text

''